In [1]:
from roboflow import Roboflow
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv('API_KEY')

rf = Roboflow(api_key=api_key)
project = rf.workspace().project("university-landmark-detection")
model = project.version(3).model

image_path = "/resized_dataset/Train/09_48_43.jpg"
# infer on a local image
print(model.predict(image_path, confidence=40, overlap=30).json())

In [3]:
import os
import json
from dotenv import load_dotenv
from roboflow import Roboflow
import cv2

# Load environment variables from .env file
load_dotenv()

# Access environment variables
api_key = os.getenv('API_KEY')
if not api_key:
    raise ValueError("API_KEY not found. Please ensure it's correctly set in the .env file.")

# Initialize Roboflow with the API key
rf = Roboflow(api_key=api_key)

# Access the specific project and model version
project_name = "university-landmark-detection"
version_number = 3

try:
    project = rf.workspace().project(project_name)
    model = project.version(version_number).model
except Exception as e:
    raise ValueError(f"Error accessing project/version: {e}")

# Define the path to the folder containing images
folder_path = "Test_2/captured_images_test2"
output_coco_file = "Test_2/annotations_coco.json"

# Create a COCO dataset structure
coco_dataset = {
    "images": [],
    "annotations": [],
    "categories": []
}

# Add categories based on the provided class names
class_names = [
    "EB1", "EB2", "EB3", "FW", "HUNT", "OVAL", "building", "security-station", 
    "sign", "street-lamp", "structure", "trashcan"
]
categories = [{"id": idx + 1, "name": name} for idx, name in enumerate(class_names)]
coco_dataset["categories"].extend(categories)

image_id = 0
annotation_id = 0

# Process each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(folder_path, filename)
        
        # Run prediction on the image
        try:
            prediction = model.predict(image_path, confidence=40, overlap=30).json()
        except Exception as e:
            print(f"Error during prediction for {image_path}: {e}")
            continue
        
        # Load the image to get its dimensions
        image = cv2.imread(image_path)
        height, width, _ = image.shape
        
        # Add image info to COCO dataset
        image_info = {
            "id": image_id,
            "file_name": filename,
            "height": height,
            "width": width
        }
        coco_dataset["images"].append(image_info)
        
        # Process predictions and add annotations to COCO dataset
        for pred in prediction['predictions']:
            x = pred['x'] - pred['width'] / 2
            y = pred['y'] - pred['height'] / 2
            width = pred['width']
            height = pred['height']
            confidence = pred['confidence']
            class_name = pred['class']
            
            # Map class name to category ID
            category_id = next((cat['id'] for cat in categories if cat['name'] == class_name), None)
            if category_id is None:
                print(f"Warning: class '{class_name}' not found in categories.")
                continue
            
            annotation = {
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x, y, width, height],
                "score": confidence,
                "area": width * height,
                "iscrowd": 0
            }
            coco_dataset["annotations"].append(annotation)
            annotation_id += 1
        
        image_id += 1

# Save the COCO dataset to a JSON file
with open(output_coco_file, 'w') as f:
    json.dump(coco_dataset, f, indent=4)

print(f"COCO dataset saved to {output_coco_file}")


loading Roboflow workspace...
loading Roboflow project...
Error during prediction for Test_2/captured_images_test2/396_darker.jpg: HTTPSConnectionPool(host='detect.roboflow.com', port=443): Max retries exceeded with url: /university-landmark-detection/3?api_key=uucie5AkN9CBB31xOHbp&name=YOUR_IMAGE.jpg&overlap=30&confidence=40&stroke=1&labels=false&format=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x747aceaac160>: Failed to resolve 'detect.roboflow.com' ([Errno -3] Temporary failure in name resolution)"))
COCO dataset saved to Test_2/annotations_coco.json
